<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240203-%E5%BE%97%E5%88%B02470%E6%A0%B7%E6%9C%AC%E7%9A%842022NDVI%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97(3.10-9.10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [30]:
import os
import pandas as pd
import math

In [31]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/randomsample_2022_470")

In [32]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

2646


In [33]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')


# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [34]:
# 5天序列NDVI
interval = 5
increment = 'day'
start = '2022-03-10'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-09-11').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDVI(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('NDVI'))

fitteds2 = s2makeup.map(add_fittedNDVI)

HANTS = fitteds2.select('NDVI')

In [35]:
# NDVI时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI', '25_NDVI', '26_NDVI', '27_NDVI', '28_NDVI', '29_NDVI', '30_NDVI', '31_NDVI', '32_NDVI', '33_NDVI', '34_NDVI', '35_NDVI', '36_NDVI', '37_NDVI']


In [36]:
image_agrgt_clip = image_agrgt.clip(roi)

In [37]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id'],scale=10,tileScale=8)

In [38]:
cols = ['id', '0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '17_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI', '25_NDVI', '26_NDVI', '27_NDVI', '28_NDVI', '29_NDVI', '30_NDVI', '31_NDVI', '32_NDVI', '33_NDVI', '34_NDVI', '35_NDVI', '36_NDVI', '37_NDVI']
inds = range(samples_datesets.size().getInfo())

In [39]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,0_NDVI,1_NDVI,2_NDVI,3_NDVI,4_NDVI,5_NDVI,6_NDVI,7_NDVI,8_NDVI,...,28_NDVI,29_NDVI,30_NDVI,31_NDVI,32_NDVI,33_NDVI,34_NDVI,35_NDVI,36_NDVI,37_NDVI
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(39),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,460,461,462,463,464,465,466,467,468,469
id,2045.000000,2046.000000,2047.000000,2048.000000,2049.000000,2050.000000,2051.000000,2052.000000,2053.000000,2054.000000,...,2461.000000,2462.000000,2463.000000,2464.000000,2465.000000,2466.000000,2467.000000,2468.000000,2469.000000,2470.000000
0_NDVI,0.418961,0.328092,0.362072,0.342298,0.311870,0.385603,0.285562,0.521376,0.307121,0.327735,...,0.589552,0.622898,0.589849,0.412943,0.479574,0.551056,0.413400,0.598613,0.645606,0.473919
1_NDVI,0.429242,0.335530,0.369157,0.348147,0.320789,0.395651,0.290765,0.524135,0.317827,0.328926,...,0.596909,0.625836,0.595673,0.417586,0.484427,0.558038,0.414150,0.606863,0.649929,0.481544
2_NDVI,0.439721,0.343146,0.377596,0.354622,0.330432,0.406651,0.296671,0.527467,0.329583,0.330517,...,0.604279,0.628373,0.601210,0.423239,0.489729,0.564853,0.415759,0.615096,0.653766,0.489121
3_NDVI,0.450324,0.350879,0.387329,0.361674,0.340728,0.418525,0.303241,0.531350,0.342308,0.332490,...,0.611610,0.630497,0.606422,0.429860,0.495450,0.571455,0.418215,0.623261,0.657099,0.496593


In [41]:
df.to_csv('sample_NDVI_2022_470.csv',encoding='utf-8')